This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform('local')

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


In [2]:
class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def bark(self):
        print(f"{self.name} says Woof!")

# Creating an instance of the Dog class
my_dog = Dog(name="Buddy", age=3)

# Accessing attributes and calling methods of the object
print(my_dog.name)  # Output: Buddy
my_dog.bark()       # Output: Buddy says Woof!

Buddy
Buddy says Woof!


This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [3]:
base = message_ix.Scenario(mp, model='GENIE_sandbox', scenario="add_tech_emission_bound1000")



Creating basic dataframe to be filled in later

In [4]:
def get_values(scenario,
               variable = '', valuetype = 'lvl',
               #filters = {}
              ): 
    # filters must use 'cat_tec' to aggregate technology
    # don't forget to include check unit
    """
    Parameters
    ----------
    scenario    : message_ix.Scenario()
        MESSAGEix Scenario where the data will be included
    variable    : string
        name of variable to report
    valuetype   : string, 'lvl' or 'mrg'
        type of values reported to report,
        either level or marginal.
        default is 'lvl'
    """
    
    if isinstance(scenario.var(variable), pd.DataFrame): # this is specific for technology, hence all should be dataframe already
        df = scenario.var(variable)
        dimensions = [col for col in df.columns if col not in ['lvl','mrg']]
        return df.set_index(dimensions)[[valuetype]]
    else:
        return scenario.var(variable)[valuetype]



In [5]:
groups = {'DACs': ['LT_DAC','HT_DAC']}

def get_report(scenario,
               grouptec = '',
              ): 
    """
    Parameters
    ----------
    scenario    : message_ix.Scenario()
        MESSAGEix Scenario where the data will be included
    grouptec    : string
        type of values reported to report,
        either level or marginal.
        default is 'lvl'
    variable    : string
        name of variable to report
    """
    var_dict = {var: [] for var in ['CAP','CAP_NEW']}
    
    for var in var_dict.keys():
        df = (get_values(scenario,var)['lvl'].unstack()
              .loc[:,groups.get(grouptec),:]
              .groupby(['node_loc']).sum()
             )
        df.loc['World'] = df.sum(axis=0)
        
        var_dict[var] = df
        
        return print(df) # TODO: This need to be printing to excel
    

In [6]:
get_values(base,'CAP')

lvl
node_loc technology year_vtg year_act            
R11_AFR  LNG_exp    1995     1995        0.000000
                    2000     2000        0.000000
                             2020        0.000000
                    2005     2005        0.000000
                             2020        8.000000
...                                           ...
R11_WEU  LT_DAC     2090     2100      218.896243
                             2110      109.448121
                    2100     2100       61.280695
                             2110       61.280695
                    2110     2110      230.610939

[124384 rows x 1 columns]

In [7]:
get_report(base,'DACs')

year_act  1960  1965  1970  1975  1980  1985  1990  1995  2000  2005  ...  \
node_loc                                                              ...   
R11_AFR    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_CPA    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_EEU    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_FSU    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_LAM    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_MEA    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_NAM    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_PAO    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_PAS    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_SAS    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
R11_WEU    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

In [8]:
mp.close_db()